In [1]:
import pandas as pd
import csv
import os
import requests
import urllib3
import csv
import time
import re
import numpy as np
# from random import randint
# from pandas import json_normalize
import censusgeocode as cg
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
from datetime import datetime
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')

In [2]:
voter = pd.read_csv('20220516_voter_race_location.csv', low_memory = False)
voter1 = voter[pd.isna(voter.lat) == False].copy()

In [3]:
regicnt = pd.DataFrame(voter1.REGISTRATION_NUMBER.value_counts())

In [4]:
regicnt[regicnt.REGISTRATION_NUMBER != 1].head()

,REGISTRATION_NUMBER


In [5]:
addrdriving = pd.read_csv('20220521_voter_address_driving_isochrone_join.csv')
addrtransit = pd.read_csv('20220523_voter_addr_transitchrone_join.csv')

In [6]:
addrdriving.head()

,Unnamed: 0,st_address,RESIDENCE_CITY,RESIDENCE_STATE,ZIP5,lat_left,long_left,index_right,travel_tim,lat_right,long_right,GEOID,in_2022,in_nov_202
0,0,237 MITCHELL RD,MAYSVILLE,GA,30558,34.267868,-83.562004,257.0,1200.0,34.335577,-83.497513,13011.0,1.0,1.0
1,0,237 MITCHELL RD,MAYSVILLE,GA,30558,34.267868,-83.562004,258.0,1800.0,34.335577,-83.497513,13011.0,1.0,1.0
2,0,237 MITCHELL RD,MAYSVILLE,GA,30558,34.267868,-83.562004,259.0,900.0,34.335577,-83.497513,13011.0,1.0,1.0
3,1,467 HICKORY CREEK RD,MAYSVILLE,GA,30558,34.308254,-83.548330,257.0,1200.0,34.335577,-83.497513,13011.0,1.0,1.0
4,1,467 HICKORY CREEK RD,MAYSVILLE,GA,30558,34.308254,-83.548330,258.0,1800.0,34.335577,-83.497513,13011.0,1.0,1.0


In [7]:
addrgeo = pd.read_csv('20220515_voter_address_tract_ua.csv')

In [8]:
longmax = pd.DataFrame(addrgeo.groupby(['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']).long_left.max())
longmax['keep'] = True
longmax = longmax.reset_index()

In [9]:
len(longmax) - len(addrgeo)

-1558

In [10]:
addrgeo1 = pd.merge(addrgeo, longmax, on = ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5','long_left'], how = 'left' )

In [11]:
len(addrgeo1[addrgeo1.keep == True])- len(addrgeo1)

-1534

In [12]:
addrgeo2 = addrgeo1[addrgeo1.keep == True].copy().drop('keep', axis = 1)

In [13]:
addrgeo1.head()

,Unnamed: 0,st_address,RESIDENCE_CITY,RESIDENCE_STATE,ZIP5,lat_left,long_left,urban,index_right,STATEFP,...,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,keep
0,0,237 MITCHELL RD,MAYSVILLE,GA,30558,34.267868,-83.562004,NaN,869.0,13.0,...,1.301197e+10,9703.0,Census Tract 9703,G5020,S,117917493.0,1953987.0,34.319561,-83.584094,True
1,1,467 HICKORY CREEK RD,MAYSVILLE,GA,30558,34.308254,-83.548330,NaN,869.0,13.0,...,1.301197e+10,9703.0,Census Tract 9703,G5020,S,117917493.0,1953987.0,34.319561,-83.584094,True
2,2,733 CARSON SEGARS RD,MAYSVILLE,GA,30558,34.295063,-83.550200,NaN,869.0,13.0,...,1.301197e+10,9703.0,Census Tract 9703,G5020,S,117917493.0,1953987.0,34.319561,-83.584094,True
3,3,11 W RIDGEWAY RD,MAYSVILLE,GA,30558,34.257618,-83.548480,NaN,869.0,13.0,...,1.301197e+10,9703.0,Census Tract 9703,G5020,S,117917493.0,1953987.0,34.319561,-83.584094,True
4,4,262 HIGHWAY 52,MAYSVILLE,GA,30558,34.261142,-83.575232,NaN,869.0,13.0,...,1.301197e+10,9703.0,Census Tract 9703,G5020,S,117917493.0,1953987.0,34.319561,-83.584094,True


In [14]:
addrgeo2 = addrgeo2.drop('Unnamed: 0', axis = 1).drop_duplicates()

In [15]:
len(addrgeo2)-len(addrgeo1)

-1534

In [16]:
latmax = pd.DataFrame(addrgeo2.groupby(['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']).lat_left.max())
latmax['keep'] = True
latmax = latmax.reset_index()

In [17]:
latmax = latmax.drop_duplicates()

In [18]:
len(latmax) - len(addrgeo2)

-24

In [19]:
addrgeo3 = pd.merge(addrgeo2, latmax, on = ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5','lat_left'], how = 'left' )

In [20]:
len(addrgeo3[addrgeo3.keep == True])- len(addrgeo3)

0

Inspecting on ArcGIS shows some of these addresses are on the edges of census tracts. deduplicaing them.<br>
and also consider the 13170 freemanville address two addresses share the same cooridnate- is that indexed to multiple voters? Should this be pruned?

In [21]:
latcount = pd.DataFrame(addrgeo2.groupby(['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']).lat_left.count())

In [22]:
latcount.head()

,,,,lat_left
st_address,RESIDENCE_CITY,RESIDENCE_STATE,ZIP5,
1 1 COURTHOUSE SQ,MCDONOUGH,GA,30253,1
1 10TH TER,TYBEE ISLAND,GA,31328,1
1 11TH AIRBORNE DIV RD,FORT BENNING,GA,31905,1
1 11TH TER,TYBEE ISLAND,GA,31328,1
1 13TH LN,TYBEE ISLAND,GA,31328,1


In [23]:
latcount[latcount.lat_left != 1].info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 24 entries, ('1297 WESLEYAN DR', 'MACON', 'GA', 31210) to ('911 NEW FRANKLIN RD', 'LAGRANGE', 'GA', 30240)
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   lat_left  24 non-null     int64
dtypes: int64(1)
memory usage: 23.8+ MB


In [24]:
tractmax = pd.DataFrame(addrgeo2.groupby(['lat_left','long_left']).TRACTCE.max())

In [25]:
tractmax.head()

,,TRACTCE
lat_left,long_left,
30.359767,-82.162196,10200.0
30.359843,-82.162622,10200.0
30.359877,-82.162928,10200.0
30.359960,-82.160714,10200.0
30.359978,-82.164920,10200.0


In [26]:
len(tractmax) - len(addrgeo2)

-62820

In [27]:
tractmax['keep'] = True

In [28]:
addrgeo3 = addrgeo3.drop('keep', axis = 1)

In [29]:
tractmax = tractmax.reset_index()

In [30]:
tractmax.head()

,lat_left,long_left,TRACTCE,keep
0,30.359767,-82.162196,10200.0,True
1,30.359843,-82.162622,10200.0,True
2,30.359877,-82.162928,10200.0,True
3,30.359960,-82.160714,10200.0,True
4,30.359978,-82.164920,10200.0,True


In [31]:
addrgeo4 = pd.merge(addrgeo3, tractmax, on = ['lat_left','long_left','TRACTCE'], how = 'outer')

In [32]:
addrgeo4[addrgeo4.keep != True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 45418 to 2998882
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   st_address       24 non-null     object 
 1   RESIDENCE_CITY   24 non-null     object 
 2   RESIDENCE_STATE  24 non-null     object 
 3   ZIP5             24 non-null     int64  
 4   lat_left         24 non-null     float64
 5   long_left        24 non-null     float64
 6   urban            20 non-null     object 
 7   index_right      24 non-null     float64
 8   STATEFP          24 non-null     float64
 9   COUNTYFP         24 non-null     float64
 10  TRACTCE          24 non-null     float64
 11  GEOID            24 non-null     float64
 12  NAME             24 non-null     float64
 13  NAMELSAD         24 non-null     object 
 14  MTFCC            24 non-null     object 
 15  FUNCSTAT         24 non-null     object 
 16  ALAND            24 non-null     float64
 17  AWATER   

In [33]:
addrgeo5 = addrgeo4[addrgeo4.keep == True].copy()

In [34]:
addrgeo = addrgeo5.copy()

In [35]:
del addrgeo1
del addrgeo2

In [36]:
addr = voter1[['lat','long']].drop_duplicates()

### load transit data

In [37]:
addrtransit1 = addrtransit.drop_duplicates()

solve for the shortes transit time bracket for an address in a given year

In [38]:
addrtransit1_22 = addrtransit1[addrtransit1.in_2022 ==  1].copy()
addrtransit1_22cnt = pd.DataFrame(addrtransit1_22.groupby([ 'st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']).travel_tim.min())
# addrtransit1_22cnt = addrtransit1_22cnt.reset_index()
addrtransit1_22cnt = addrtransit1_22cnt.rename(columns = {'travel_tim':'min_transit_2022'})

In [39]:
addrtransit1_20 = addrtransit1[addrtransit1.in_nov_202 ==  1].copy()
addrtransit1_20cnt = pd.DataFrame(addrtransit1_20.groupby([ 'st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']).travel_tim.min())
# addrtransit1_20cnt = addrtransit1_20cnt.reset_index()
addrtransit1_20cnt = addrtransit1_20cnt.rename(columns = {'travel_tim':'min_transit_2020'})

In [40]:
addtransitmaster = pd.merge(addrtransit1_20cnt, addrtransit1_22cnt, left_index = True, right_index = True, how = 'outer')

In [41]:
addtransitmaster['band_diff'] = addtransitmaster.min_transit_2022 - addtransitmaster.min_transit_2020
addtransitmaster = addtransitmaster.reset_index()
addtransitmaster.ZIP5 = [str(x) for x in addtransitmaster.ZIP5]

In [42]:
addtransitmaster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259084 entries, 0 to 259083
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   st_address        259084 non-null  object 
 1   RESIDENCE_CITY    259084 non-null  object 
 2   RESIDENCE_STATE   259084 non-null  object 
 3   ZIP5              259084 non-null  object 
 4   min_transit_2020  243056 non-null  float64
 5   min_transit_2022  93558 non-null   float64
 6   band_diff         77530 non-null   float64
dtypes: float64(3), object(4)
memory usage: 13.8+ MB


In [43]:
votertransit = pd.merge(voter1,addtransitmaster, on =  ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5'], how = 'left')

In [44]:
votertransit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7528561 entries, 0 to 7528560
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Unnamed: 0           int64  
 1   REGISTRATION_NUMBER  int64  
 2   BIRTHDATE            int64  
 3   RACE                 object 
 4   st_address           object 
 5   RESIDENCE_CITY       object 
 6   RESIDENCE_STATE      object 
 7   ZIP5                 object 
 8   lat                  float64
 9   long                 float64
 10  min_transit_2020     float64
 11  min_transit_2022     float64
 12  band_diff            float64
dtypes: float64(5), int64(3), object(5)
memory usage: 804.1+ MB


In [45]:
len(votertransit)/len(votertransit.REGISTRATION_NUMBER.unique())

1.0

In [46]:
votertransittest = pd.merge(voter1,addtransitmaster, on =  ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5'], how = 'outer')

In [47]:
# chcecking that the merge was right
len(votertransittest) == len(votertransit)

True

In [48]:
del votertransittest

In [49]:
len(votertransit.REGISTRATION_NUMBER.unique())/len(votertransit)

1.0

In [50]:
len(votertransit)

7528561

In [51]:
len(votertransit[pd.isna(votertransit.min_transit_2020) == False])

741656

In [52]:
len(votertransit[votertransit.RACE == 'WH'].REGISTRATION_NUMBER.unique())

3935160

### load vehicle ownership data

In [53]:
carraw = pd.read_csv('ACS_household_vehicle_by_tract_ACSDP5Y2020.DP04_2022-05-15T192441/ACSDP5Y2020.DP04_data_with_overlays_2022-05-15T180221.csv', low_memory = False)

In [54]:
carraw = carraw.drop(0)

In [55]:
car = carraw[['GEO_ID','DP04_0058PE']].copy()

In [56]:
car = car.rename(columns = {'DP04_0058PE':'pct_no_car'})

In [57]:
car.head()

,GEO_ID,pct_no_car
1,1400000US13001950100,5.1
2,1400000US13001950201,4.0
3,1400000US13001950202,13.1
4,1400000US13001950301,4.4
5,1400000US13001950302,32.5


In [58]:
car['GEOID'] = [x.split('US')[1] for x in car.GEO_ID]

In [59]:
car = car.drop('GEO_ID', axis = 1)

In [60]:
addrgeo.ZIP5 = [str(x) for x in addrgeo.ZIP5 ]

In [61]:
addrgeo.GEOID = [str(int(x)) if pd.isna(x) == False else str(x) for x in addrgeo.GEOID]

In [62]:
addrgeo1 = pd.merge(addrgeo, car, on = 'GEOID', how = 'left')

In [63]:
addrgeo1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3190879 entries, 0 to 3190878
Data columns (total 22 columns):
 #   Column           Dtype  
---  ------           -----  
 0   st_address       object 
 1   RESIDENCE_CITY   object 
 2   RESIDENCE_STATE  object 
 3   ZIP5             object 
 4   lat_left         float64
 5   long_left        float64
 6   urban            object 
 7   index_right      float64
 8   STATEFP          float64
 9   COUNTYFP         float64
 10  TRACTCE          float64
 11  GEOID            object 
 12  NAME             float64
 13  NAMELSAD         object 
 14  MTFCC            object 
 15  FUNCSTAT         object 
 16  ALAND            float64
 17  AWATER           float64
 18  INTPTLAT         float64
 19  INTPTLON         float64
 20  keep             object 
 21  pct_no_car       object 
dtypes: float64(11), object(11)
memory usage: 559.9+ MB


In [64]:
votertransit1 = pd.merge(votertransit, addrgeo1, on = ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5'], how = 'left')

In [65]:
votertransit1.pct_no_car = [np.nan if x == '-' else float(x) for x in votertransit1.pct_no_car]

In [66]:
len(votertransit1.REGISTRATION_NUMBER.unique())-len(votertransit1)

0

### load driving data

In [67]:
addrdriving1 = addrdriving.drop_duplicates()

solve for the shortes driving time bracket for an address in a given year

In [68]:
addrdriving1_22 = addrdriving1[addrdriving1.in_2022 ==  1].copy()
addrdriving1_22cnt = pd.DataFrame(addrdriving1_22.groupby([ 'st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']).travel_tim.min())
# addrdriving1_22cnt = addrdriving1_22cnt.reset_index()
addrdriving1_22cnt = addrdriving1_22cnt.rename(columns = {'travel_tim':'min_driving_2022'})

In [69]:
addrdriving1_20 = addrdriving1[addrdriving1.in_nov_202 ==  1].copy()
addrdriving1_20cnt = pd.DataFrame(addrdriving1_20.groupby([ 'st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']).travel_tim.min())
# addrdriving1_20cnt = addrdriving1_20cnt.reset_index()
addrdriving1_20cnt = addrdriving1_20cnt.rename(columns = {'travel_tim':'min_driving_2020'})

In [70]:
adddrivingmaster = pd.merge(addrdriving1_20cnt, addrdriving1_22cnt, left_index = True, right_index = True, how = 'outer')

In [71]:
adddrivingmaster['band_diff'] = adddrivingmaster.min_driving_2022 - adddrivingmaster.min_driving_2020

In [72]:
adddrivingmaster = adddrivingmaster.reset_index()

In [73]:
adddrivingmaster.ZIP5 = [str(x) for x in adddrivingmaster.ZIP5]

In [74]:
adddrivingmaster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172094 entries, 0 to 3172093
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   st_address        object 
 1   RESIDENCE_CITY    object 
 2   RESIDENCE_STATE   object 
 3   ZIP5              object 
 4   min_driving_2020  float64
 5   min_driving_2022  float64
 6   band_diff         float64
dtypes: float64(3), object(4)
memory usage: 169.4+ MB


In [75]:
voterdriving = pd.merge(voter1,adddrivingmaster, on =  ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5'], how = 'left')

In [76]:
voterdriving = voterdriving.rename(columns = {'band_diff':'driving_band_diff'})

In [77]:
votertransit2 = votertransit1.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,15,16,19,20,21,30]].copy().drop_duplicates()

In [78]:
votertransit2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7528561 entries, 0 to 7528560
Data columns (total 18 columns):
 #   Column               Dtype  
---  ------               -----  
 0   REGISTRATION_NUMBER  int64  
 1   BIRTHDATE            int64  
 2   RACE                 object 
 3   st_address           object 
 4   RESIDENCE_CITY       object 
 5   RESIDENCE_STATE      object 
 6   ZIP5                 object 
 7   lat                  float64
 8   long                 float64
 9   min_transit_2020     float64
 10  min_transit_2022     float64
 11  band_diff            float64
 12  urban                object 
 13  index_right          float64
 14  TRACTCE              float64
 15  GEOID                object 
 16  NAME                 float64
 17  pct_no_car           float64
dtypes: float64(9), int64(2), object(7)
memory usage: 1.1+ GB


In [79]:
len(votertransit2)/len(votertransit1)

1.0

In [80]:
votertransit2 = votertransit2.rename(columns = {'band_diff':'transit_band_diff'})

In [81]:
traveltimemaster = pd.merge(voterdriving, votertransit2, on = ['REGISTRATION_NUMBER','RACE','st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5','lat','long'], how = 'outer')

In [82]:
len(traveltimemaster)/len(voterdriving)

1.0

In [83]:
len(traveltimemaster)/len(votertransit2)

1.0

In [84]:
len(voterdriving)

7528561

In [85]:
def compareval(x,y):
    if pd.isna(x) == True:
        return(y)
    if pd.isna(y) == True:
        return (x)
    else:
        if x >= y:
            return (y)
        else:
            return (x)

In [86]:
traveltimemaster['min_travel_time_2020'] = [ compareval(x,y) for (x,y) in zip(traveltimemaster.min_driving_2020, traveltimemaster.min_transit_2020)]

In [87]:
traveltimemaster.min_travel_time_2020.value_counts().sum()

6862276

In [88]:
traveltimemaster['min_travel_time_2022'] = [compareval(x,y) for (x,y) in zip(traveltimemaster.min_driving_2022, traveltimemaster.min_transit_2022)]

In [89]:
traveltimemaster.min_travel_time_2022.value_counts().sum()

7458177

In [90]:
traveltimemaster.min_travel_time_2022.value_counts().sum() - traveltimemaster.min_travel_time_2020.value_counts().sum()

595901

In [91]:
def get_diff(old,new):
    if (pd.isna(old) == True) & (pd.isna(new) == False):
        return(1)
    if (pd.isna(old) == False) & (pd.isna(new) == True):
        return(-1)
    if (pd.isna(old) == True) & (pd.isna(new) == True):
        return(np.nan)
    if (pd.isna(old) == False) & (pd.isna(new) == False):
        return(old - new)

In [92]:
traveltimemaster['travel_diff'] = [get_diff(x,y) for (x,y) in zip(traveltimemaster.min_travel_time_2020, traveltimemaster.min_travel_time_2022)]

In [93]:
traveltimemaster.urban = [False if pd.isna(x) == True else x for x in traveltimemaster.urban]

In [94]:
income = pd.read_csv('median_household_income_ACSDT5Y2020.B19013_2022-05-24T100026/ACSDT5Y2020.B19013_data_with_overlays_2022-05-24T100020.csv')

In [95]:
income = income.drop(0)
income = income.drop(['NAME','B19013_001M'], axis = 1)
income['GEOID'] = [x.split('US')[1] for x in income.GEO_ID]
income = income.drop('GEO_ID', axis = 1)
income = income.rename(columns = {'B19013_001E':'median_household_income'})

In [96]:
traveltimemaster1 = pd.merge(traveltimemaster,income, on = 'GEOID', how = 'left')

In [97]:
def clean_income(x):
    if x == '-':
        return(np.nan)
    if x == '250,000+':
        return (250000)
    else:
        return(float(x))

In [98]:
traveltimemaster1['income_cleaned'] = [clean_income(x) for x in traveltimemaster1.median_household_income]

In [99]:
import mapclassify

In [100]:
incomebins = mapclassify.Quantiles(traveltimemaster1[pd.isna(traveltimemaster1.income_cleaned) == False].income_cleaned, k=4).bins

In [101]:
def get_qt(num):
    if num <= incomebins[0]:
        return 1
    else:
        if (num > incomebins[0]) & (num <= incomebins[1]):
            return (2)
        else:
            if (num > incomebins[1]) & (num <= incomebins[2]):
                return(3)
            else:
                if(num > incomebins[2]) & (num <= incomebins[3]):
                    return(4)
                else:
                    return(np.nan)


In [102]:
traveltimemaster1['income_qt'] = [get_qt(x) for x in traveltimemaster1.income_cleaned]

In [103]:
age = pd.read_csv('20220526_voter_age.csv')

In [104]:
traveltimemaster2 = pd.merge(traveltimemaster1, age, on = 'REGISTRATION_NUMBER', how = 'left')

In [105]:
traveltimemaster2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7528561 entries, 0 to 7528560
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0_x             int64  
 1   REGISTRATION_NUMBER      int64  
 2   BIRTHDATE_x              int64  
 3   RACE                     object 
 4   st_address               object 
 5   RESIDENCE_CITY           object 
 6   RESIDENCE_STATE          object 
 7   ZIP5                     object 
 8   lat                      float64
 9   long                     float64
 10  min_driving_2020         float64
 11  min_driving_2022         float64
 12  driving_band_diff        float64
 13  BIRTHDATE_y              int64  
 14  min_transit_2020         float64
 15  min_transit_2022         float64
 16  transit_band_diff        float64
 17  urban                    bool   
 18  index_right              float64
 19  TRACTCE                  float64
 20  GEOID                    object 
 21  NAME    

In [106]:
len(traveltimemaster1)/len(traveltimemaster2)

1.0

adding precinct voting data

In [107]:
precinct = gpd.read_file('precincts-with-results.geojson/precincts-with-results.geojson')

In [108]:
traveltimemaster2['county_fips'] = [x[:5] for x in traveltimemaster2.GEOID]

In [109]:
GAcounty = list(traveltimemaster2.county_fips.unique())

In [110]:
len(GAcounty)

160

In [111]:
precinct['county_fips'] = [x.split('-')[0] for x in precinct.GEOID ]

In [112]:
GAprecinct = precinct[precinct.county_fips.isin(GAcounty)].copy()

In [113]:
del precinct

In [114]:
GAprecinct[GAprecinct.pct_dem_lead == 0].info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4 entries, 33770 to 35708
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   GEOID           4 non-null      object  
 1   votes_dem       4 non-null      float64 
 2   votes_rep       4 non-null      float64 
 3   votes_total     4 non-null      float64 
 4   votes_per_sqkm  4 non-null      float64 
 5   pct_dem_lead    4 non-null      float64 
 6   geometry        4 non-null      geometry
 7   county_fips     4 non-null      object  
dtypes: float64(5), geometry(1), object(2)
memory usage: 288.0+ bytes


In [115]:
def parse_party(x):
    if x == 0:
        return('tie')
    if x > 0:
        return('D')
    if x < 0:
        return('R')
    else:
        return(np.nan)

In [116]:
GAprecinct['party'] = [parse_party(x) for x in GAprecinct.pct_dem_lead]

In [117]:
GAprecinct1 = GAprecinct[['GEOID','geometry','party']].copy()

In [118]:
traveltimemaster3 = gpd.GeoDataFrame(traveltimemaster2, geometry=gpd.points_from_xy(traveltimemaster2.long,traveltimemaster2.lat))

In [119]:
traveltimemaster3.crs = 'EPSG:4326'

In [120]:
traveltimemaster3 = traveltimemaster3.drop('index_right', axis = 1)

In [121]:
len(traveltimemaster3) - len(traveltimemaster2)

0

In [122]:
traveltimemaster4 = gpd.sjoin(traveltimemaster3, GAprecinct1, how ='left')

In [124]:
GAprecinct1.head()

,GEOID,geometry,party
33623,13001-5B,"POLYGON ((-82.39899 31.70246, -82.39968 31.701...",R
33624,13001-4B,"POLYGON ((-82.25807 31.67926, -82.25736 31.679...",R
33625,13001-3C,"POLYGON ((-82.13442 31.67122, -82.13657 31.671...",R
33626,13001-1B,"POLYGON ((-82.43015 31.96599, -82.42865 31.965...",R
33627,13001-4D,"POLYGON ((-82.42831 31.66111, -82.42682 31.662...",R


In [125]:
len(traveltimemaster4)-len(traveltimemaster3)

19

the default is intersect so probably some addresses intersected with mulitple geoids. picking the first one

In [126]:
maxprecinct = pd.DataFrame(traveltimemaster4.groupby(['lat','long']).GEOID_right.first())

In [127]:
maxprecinct['precint_keep'] = True

In [128]:
maxprecinct = maxprecinct.reset_index()
maxprecinct = maxprecinct.drop_duplicates().copy()

In [129]:
len(maxprecinct)

3128083

In [136]:
maxprecinct.head()

,lat,long,GEOID_right,precint_keep
0,30.359767,-82.162196,13049-GEORGIA BEND FIRE STATION (3B),True
1,30.359843,-82.162622,13049-GEORGIA BEND FIRE STATION (3B),True
2,30.359877,-82.162928,13049-GEORGIA BEND FIRE STATION (3B),True
3,30.359960,-82.160714,13049-GEORGIA BEND FIRE STATION (3B),True
4,30.359978,-82.164920,13049-GEORGIA BEND FIRE STATION (3B),True


In [130]:
traveltimemaster4.lat = np.round(traveltimemaster4.lat,6)
traveltimemaster4.long = np.round(traveltimemaster4.long,6)

In [131]:
maxprecinct.lat = np.round(maxprecinct.lat,6)
maxprecinct.long = np.round(maxprecinct.long,6)

In [178]:
maxprecinct = maxprecinct.drop_duplicates()

In [181]:
len(maxprecinct)

3127203

In [184]:
del traveltimemaster
del traveltimemaster1
del traveltimemaster2
del traveltimemaster3

In [185]:
traveltimemaster5 = traveltimemaster4.merge(maxprecinct, on = ['lat','long','GEOID_right'], how = 'left')

In [188]:
len(traveltimemaster5[traveltimemaster5.precint_keep == True])-len(traveltimemaster4)

-19

In [192]:
traveltimemaster5 = traveltimemaster5[traveltimemaster5.precint_keep  == True]

In [193]:
geoidcnt = pd.DataFrame(traveltimemaster5.groupby('REGISTRATION_NUMBER').GEOID_right.count())

In [194]:
geoidcnt[geoidcnt.GEOID_right > 1].head()

,GEOID_right
REGISTRATION_NUMBER,


In [196]:
traveltimemaster5.pct_no_car.quantile(.9)

14.8

In [197]:
traveltimemaster5['no_car'] = [1 if x >= 14.8 else 0 for x in traveltimemaster5.pct_no_car]

In [198]:
traveltimemaster5['elderly'] = [1 if x>=65 else 0 for x in traveltimemaster5.age]

In [199]:
traveltimemaster5['min_travel_2020_recode'] = [1200 if x == 900 else x for x in traveltimemaster5.min_travel_time_2020]

In [200]:
traveltimemaster5['min_travel_2022_recode'] = [1200 if x == 900 else x for x in traveltimemaster5.min_travel_time_2022]

In [201]:
traveltimemaster5.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7528561 entries, 0 to 7528579
Data columns (total 40 columns):
 #   Column                   Dtype   
---  ------                   -----   
 0   Unnamed: 0_x             int64   
 1   REGISTRATION_NUMBER      int64   
 2   BIRTHDATE_x              int64   
 3   RACE                     object  
 4   st_address               object  
 5   RESIDENCE_CITY           object  
 6   RESIDENCE_STATE          object  
 7   ZIP5                     object  
 8   lat                      float64 
 9   long                     float64 
 10  min_driving_2020         float64 
 11  min_driving_2022         float64 
 12  driving_band_diff        float64 
 13  BIRTHDATE_y              int64   
 14  min_transit_2020         float64 
 15  min_transit_2022         float64 
 16  transit_band_diff        float64 
 17  urban                    bool    
 18  TRACTCE                  float64 
 19  GEOID_left               object  
 20  NAME            